<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/02_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Final_Project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 요기요 데이터셋 불러오기

In [ ]:
train = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0908.csv")
train.head()

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,[직화]삼겹옛날도시락
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요:),5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치"
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥"
3,1988응답하라추억의옛날도시락-방이점,"..,........",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,삼겹정찬도시락
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러..."


- 별점 비율 확인해보기

In [ ]:
for i in range(1, 6):
    num_rating = train[train['별점'] == i].shape[0]
    print(f"별점 {i}점 개수: {num_rating}, {num_rating / train.shape[0] * 100}%")

별점 1점 개수: 220, 0.5923532579429186%
별점 2점 개수: 137, 0.36887452880990845%
별점 3점 개수: 385, 1.0366182014001075%
별점 4점 개수: 1234, 3.3225632740980076%
별점 5점 개수: 35164, 94.67959073774905%


# 데이터 전처리

## 1_특수문자 제거

In [ ]:
review_list = []
for review in train['고객리뷰']:
    if isinstance(review, str):
        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['고객리뷰'] = review_list
train.head()

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다💖 \n\n보람차게 일할 수 있게 해 주...,[직화]삼겹옛날도시락
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요,5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치"
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다!😄 \n\n맛있는 메뉴를 넉넉하게 먹을 수 ...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥"
3,1988응답하라추억의옛날도시락-방이점,".,.",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? \n\n1. 고객을 생각하는 마음 \n2...,삼겹정찬도시락
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요!😁\n\n고객님의 행복...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러..."


In [ ]:
reply_list = []
for reply in train['사장댓글']:
    if isinstance(reply, str):
        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['사장댓글'] = reply_list
train.head()

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,[직화]삼겹옛날도시락
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요,5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치"
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥"
3,1988응답하라추억의옛날도시락-방이점,".,.",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,삼겹정찬도시락
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러..."


##2_고객ID -> 고객님

In [ ]:
train['고객리뷰'] = train['고객리뷰'].astype(str)
train['사장댓글'] = train['사장댓글'].astype(str)

In [ ]:
train['사장댓글'] = train['사장댓글'].str.replace(r'\w+\s* 님', '고객님', regex=True)

##3_띄어쓰기/맞춤법

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-ogsn_qy1
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-ogsn_qy1
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=369b5290ea33209fbb35083d991a783e5800a9ca94ecae6d73e38967052700c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-hy52o4my/wheels/2e/43/cc/753c9e1d91affb9ea40e186cea5654fb9231deb454da6724e5
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker

def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [ ]:
corrected_reply = []
for review in tqdm(train['사장댓글']):
    tmp = correct_spell(review)
    corrected_reply.append(tmp)

  0%|          | 0/37140 [00:00<?, ?it/s]

In [ ]:
train["corrected_reply"] = corrected_reply

In [ ]:
train.head()

,가게이름,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,메뉴,branch,corrected_reply
0,1988응답하라추억의옛날도시락-방이점,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,5,5.0,5.0,5.0,즐거운 순간을 저희와 함께해주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감사...,[직화]삼겹옛날도시락,,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...
1,1988응답하라추억의옛날도시락-방이점,기름 있는 부분이 생각보다 많아요! 참고하세요,5,5.0,5.0,5.0,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨 듣고 앞으로 더욱 더 개선 할 수 있...,"[직화]삼겹옛날도시락,볶음김치",,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 ...
2,1988응답하라추억의옛날도시락-방이점,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,5,5.0,5.0,5.0,늘 최고의 메뉴를 제공해드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것보...,"[직화]삼겹옛날도시락,우삼겹청국장찌개＋대접밥",,늘 최고의 메뉴를 제공해 드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것...
3,1988응답하라추억의옛날도시락-방이점,".,.",5,5.0,5.0,5.0,저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,삼겹정찬도시락,,저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...
4,1988응답하라추억의옛날도시락-방이점,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,5,5.0,4.0,5.0,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,"갈비옛날도시락＋미니냉면＋（서비스）콜라,스팸옛날도시락＋김찌 정식＋（서비스）콜라,부드러...",,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...


In [ ]:
train_df = train[['고객리뷰','corrected_reply']]
train_df.head()

,고객리뷰,corrected_reply
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 ...
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해 드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것...
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...


##4_가게명 키워드 컬럼화

In [ ]:
store_list = train['가게이름']
store_list = (store_list.str.split('-')).str.get(0)
store_list

0        1988응답하라추억의옛날도시락
1        1988응답하라추억의옛날도시락
2        1988응답하라추억의옛날도시락
3        1988응답하라추억의옛날도시락
4        1988응답하라추억의옛날도시락
               ...       
37135                 짬뽕관
37136                 짬뽕관
37137                 짬뽕관
37138                 짬뽕관
37139                 짬뽕관
Name: 가게이름, Length: 37140, dtype: object

In [ ]:
tmp = list(set(store_list.replace("[^가-힣a-zA-Z0-9 ]"," ",regex=True)))
tmp

['올데이케밥 샐러드',
 '밈피자',
 '최가네한쌈',
 '모모타코야키 카페',
 '저팔계 중국 마라탕 덮밥 요리 전문점',
 '피제이피자',
 '청년치킨',
 '땡초곱창막창',
 '이화수육개장',
 '빅스타피자',
 'OK반점',
 '고기혁명',
 '구구족',
 '프레드피자',
 '떡형',
 '안녕 닭',
 '미친피자',
 '누보닭갈비',
 '코브라독스',
 '용문',
 '행복한찜닭',
 '대찌 대파생고기김치찌개 정육왕국물두루치기',
 '부성초밥',
 '비오키친',
 '스시사소우',
 '명작파스타',
 '최고남제육명가',
 '올데이파스타',
 '탕화쿵푸 마라탕 가오점',
 '김포불닭발',
 '송파만다린',
 '닭장수섭삼계탕 전기구이통닭',
 '봉명동내커피',
 '탐나는피자',
 '짚신스시',
 '우리집반찬도시락',
 '더팔당매운오징어 닭발 김치찜',
 '본점 장충왕족발보쌈',
 '자성당',
 '이태리면가게',
 '짬뽕관',
 '1989마라탕',
 '쇼부덮밥',
 '피자이탈리',
 '떡깨비',
 '삼다수',
 '홍스족발',
 '여왕의브런치',
 '한식세끼1인김치찜 김치찌개',
 '초밥대통령',
 '39도시락',
 '꼬알라파이',
 '걸작떡볶이',
 '진지덮밥',
 '모어댄버터',
 '미락칼국수',
 '오늘은 분식',
 '병천토속순대',
 '랜돌프뉴욕페페로니',
 '과일에반하다',
 '빽다방',
 '홍싸롱',
 '배달의쌀국수',
 '상무초밥',
 '백소정',
 '콩샌전북대점',
 '손수경의육회담은연어',
 '삽교원조두리곱창',
 '어시장',
 '라화방마라탕',
 '팔도한마당순대',
 '냉면 숯불고기전문점줄벼락',
 '큰아들백순대',
 '짜글이가돼었소',
 '마라하오',
 '보돌미역',
 '더티베지',
 '맘스터치 대덕테크노벨리점',
 '마라쿡',
 '피자프라텔로',
 '부산오빠1인국밥',
 '잭아저씨족발보쌈',
 '수상한삼겹살',
 '텐텐마라탕',
 '카페봄봄',
 '전금례닭볶음탕',
 '부대찌개자신있는집',
 '수제죽전문점',
 '나인곱창',
 '킹프레소빅와

- 띄어쓰기 있을 경우 띄어쓰기 없앤 가게명 (ex)놀부부대찌개

In [ ]:
tmp2 = [''.join(store.split()) for store in tmp]
tmp2

['올데이케밥샐러드',
 '밈피자',
 '최가네한쌈',
 '모모타코야키카페',
 '저팔계중국마라탕덮밥요리전문점',
 '피제이피자',
 '청년치킨',
 '땡초곱창막창',
 '이화수육개장',
 '빅스타피자',
 'OK반점',
 '고기혁명',
 '구구족',
 '프레드피자',
 '떡형',
 '안녕닭',
 '미친피자',
 '누보닭갈비',
 '코브라독스',
 '용문',
 '행복한찜닭',
 '대찌대파생고기김치찌개정육왕국물두루치기',
 '부성초밥',
 '비오키친',
 '스시사소우',
 '명작파스타',
 '최고남제육명가',
 '올데이파스타',
 '탕화쿵푸마라탕가오점',
 '김포불닭발',
 '송파만다린',
 '닭장수섭삼계탕전기구이통닭',
 '봉명동내커피',
 '탐나는피자',
 '짚신스시',
 '우리집반찬도시락',
 '더팔당매운오징어닭발김치찜',
 '본점장충왕족발보쌈',
 '자성당',
 '이태리면가게',
 '짬뽕관',
 '1989마라탕',
 '쇼부덮밥',
 '피자이탈리',
 '떡깨비',
 '삼다수',
 '홍스족발',
 '여왕의브런치',
 '한식세끼1인김치찜김치찌개',
 '초밥대통령',
 '39도시락',
 '꼬알라파이',
 '걸작떡볶이',
 '진지덮밥',
 '모어댄버터',
 '미락칼국수',
 '오늘은분식',
 '병천토속순대',
 '랜돌프뉴욕페페로니',
 '과일에반하다',
 '빽다방',
 '홍싸롱',
 '배달의쌀국수',
 '상무초밥',
 '백소정',
 '콩샌전북대점',
 '손수경의육회담은연어',
 '삽교원조두리곱창',
 '어시장',
 '라화방마라탕',
 '팔도한마당순대',
 '냉면숯불고기전문점줄벼락',
 '큰아들백순대',
 '짜글이가돼었소',
 '마라하오',
 '보돌미역',
 '더티베지',
 '맘스터치대덕테크노벨리점',
 '마라쿡',
 '피자프라텔로',
 '부산오빠1인국밥',
 '잭아저씨족발보쌈',
 '수상한삼겹살',
 '텐텐마라탕',
 '카페봄봄',
 '전금례닭볶음탕',
 '부대찌개자신있는집',
 '수제죽전문점',
 '나인곱창',
 '킹프레소빅와플',
 '도우라이크피자',
 '써브

- 띄어쓰기한 가게명

In [ ]:
from hanspell import spell_checker

corrected_store = []
for name in tmp:
    hanspell_sent = spell_checker.check(name)
    corrected_store.append(hanspell_sent.checked)

corrected_store

['올 데이 케밥 샐러드',
 '밈이 피자',
 '최가네 한 쌈',
 '모모 다코야키 카페',
 '저팔계 중국 마라탕 덮밥 요리 전문점',
 '피제이 피자',
 '청년 치킨',
 '땡초 곱창 막창',
 '이화수 육개장',
 '빅스타 피자',
 'OK 반점',
 '고기 혁명',
 '구구족',
 '프레드 피자',
 '떡 형',
 '안녕 닭',
 '미친 피자',
 '누보 닭갈비',
 '코브라 독습',
 '용문',
 '행복한 찜닭',
 '대치 대파 생고기 김치찌개 정육 왕국 물 두루치기',
 '부성 초밥',
 '비 오키친',
 '스시가 소우',
 '명작 파스타',
 '최고 날제육 명가',
 '올 데이 파스타',
 '탕화 쿵후 마라탕 가오점',
 '김포 불 닭발',
 '송파 만다린',
 '닭 장수 섭 삼계탕 전기구이 통닭',
 '봉명동 내 커피',
 '탐나는 피자',
 '짚신 스시',
 '우리 집 반찬도시락',
 '더 팔당 매운 오징어 닭발 김치찜',
 '본점 장충 왕족발보쌈',
 '자 성당',
 '이태리면 가게',
 '짬뽕과',
 '1989마라탕',
 '승부 덮밥',
 '피자 이탈리',
 '떡 깨비',
 '삼다수',
 '홍수 족발',
 '여왕의 브런치',
 '한식 세 끼 1인 김치찜 김치찌개',
 '초밥 대통령',
 '39도시락',
 '코알라 파이',
 '걸작 떡볶이',
 '진지 덮밥',
 '모어 덴버 터',
 '미락 칼국수',
 '오늘은 분식',
 '병천 토속 순대',
 '랜돌프 뉴욕 페퍼로니',
 '과일에 반하다',
 '빽다방',
 '홍싸롱',
 '배달의 쌀국수',
 '상무 초밥',
 '백 소장',
 '콩 샌 전북대점',
 '손수경의 육회 담은 연어',
 '삽교 원조 두리 곱창',
 '어시장',
 '라화방마라탕',
 '팔도 한마당 순대',
 '냉면 숯불고기전문점줄벼락',
 '큰아들 백순대',
 '짜 글이가 되었소',
 '마라하오',
 '보도로 미역',
 '더티 베지',
 '맘스터치 대덕테크노벨리점',
 '마라 쿡',
 '피자 프라 텔로',
 '부산 오빠 1인 국밥

- 가게명 글자개수 단위로 쪼개기

In [ ]:
short_name = []
for name in store_list:
    for length in [4, 3, 2]:
        cut = name[:length]
        short_name.append(cut)
    if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
        cut = name[:6]
        short_name.append(cut)

short_name = [name for name in short_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
short_name = list(set(short_name))
short_name = sorted(short_name, reverse=True, key=len)
print(len(short_name), short_name)

448 ['1989마라', '1988응답', '7번가피자', '39도시락', '스시사소', '청년치킨', '우시장국', '코브라독', '탕화쿵푸', '닭장수섭', '과일에반', '도우라이', '승도리네', '쇼부덮밥', '이배달 ', '텐텐마라', '손수경의', '오늘은 ', '이끌림마', '띵하오차', '도야짬뽕', '부대찌개', '39도시', '헬로팬닭', '홍대개미', '손이가시', '돈까스회', '7번가피', '송파만다', '한식세끼', '나인곱창', '써브웨이', '봉명동내', '폴트버거', '인생아구', '모어댄버', '최가네한', '토핑폭탄', '김포불닭', '놀부 부', '부산오빠', '박가부대', '랜돌프뉴', '잭아저씨', '짜글이가', '명작파스', '구래상회', '뜸들이다', '큰아들백', '워커스커', '라화방마', '깨돌이김', '직화삼겹', '춘리마라', '올데이파', '우리집반', '걸작떡볶', '유쾌한꼬', '피자스웨', '홍스족발', '원할머니', '킹프레소', '순살만공', '더팔당매', '보돌미역', '카페봄봄', '짱닭치킨', '국수나무', '땅끝수산', '꼬알라파', '대한냉면', '혼밥대왕', '갈비민족', '행복한찜', '황궁쟁반', '빕스얌딜', '안녕,닭', '병천토속', '맘스터치', '비오키친', '야키토리', '고기잔뜩', '팔도한마', '올데이케', '상무초밥', '놀부보쌈', '여왕의브', '청춘식당', '삽교원조', '마라하오', '플러스8', '피제이피', '수제죽전', '탐나는피', '신자매김', '피자이탈', '프레드피', '모모타코', '이태리면', '울진죽변', '땡초곱창', '창타이누', '이화수육', '피자보이', '이토우야', '저팔계&', '피자프라', '전금례닭', '호랑이초', '공주닭발', '인생제육', 'GTS버', 'OK반점', '고기혁명', '미친피자', '천년닭강', '육회&연', '부성초밥', '초밥대통', '본점 장', '짚신스시', '대찌 대', '조형훈족', '빅스타피

In [ ]:
# short_name 에서 가게명 이외 일상용어로 사용되는 키워드는 제거
drop_name = ['오늘은 ', '고기잔뜩', '돈까스회', '부대찌개', '야키토리', '한식세끼', '이태리면', '모어댄버', '손이가시', '부산오빠', '고기듬뿍', '이태리', '손이가', '부산오', '저팔계', '수제죽', '청춘식', '피자이', '수상한', '안녕,', '행복한', '피자보', '탐나는', '오늘은', '짜글이', '이화수', '울진죽', '피자스', '여왕의', '배달의', '플러스', '봉명동', '최가네', '우리집', '과일에', '인생아', '호랑이', '국수나', '미친피', '뜸들이', '이배달', '어시장', '본점 ', '이끌림', '삼다수', '유쾌한', '명작파', '놀부 ', '돈까스', '큰아들', '구구족', '39도', '육회&', '빕스얌', '송파만', '대찌 ', '감성커', '상무초', '시민보', '땅끝수', '올데이', '홍대개', '모어댄', '혼밥대', '손이', '팔도', '명작', 'GT', '모모', '이화', '당치', '수락', '땅끝', '이끌', '청년', '신자', '짬뽕', '상무', '유쾌', '짱닭', '오늘', '더티', '수상', '닭장', '봉명', '부성', '행복', '놀부', '빕스', '누보', '도야', '이태', '왕빈', '미친', '저팔', '청춘', '전금', '박가', '어시', '부대', '토핑', '감성', '순살', '빅스', '고씨', '떡형', '대한', '콩샌', '쇼부', '이배', '대찌', '최가', '자성', '탐나', '플러', '우리', '부산', '진지', '한식', '여왕', '스시', '원할', '승도', 'OK', '더팔', '초밥', '모어', '송파', '카페', '김포', '고돼', '땡초', '시민', '배달', '고기', '홍대', '혼밥', '잭아', '삽교', '병천', '워커', '직화', '큰아', '홍싸', '후스', '도우', '무국', '홍스', '본점', '안녕', '육회', '인생', '냉면', '짚신', '황궁', '울진', '손수', '과일', '피자', '갈비', '조형', '걸작', '공주', '천년', '수제', '나인', '삼다', '써브', '7번', '국수', '헬로', '구구', '야키', '뜸들', '구래', '우시', '최고']
short_name = [col for col in short_name if col not in drop_name]
print(len(short_name), short_name)

271 ['1989마라', '1988응답', '7번가피자', '39도시락', '스시사소', '청년치킨', '우시장국', '코브라독', '탕화쿵푸', '닭장수섭', '과일에반', '도우라이', '승도리네', '쇼부덮밥', '이배달 ', '텐텐마라', '손수경의', '이끌림마', '띵하오차', '도야짬뽕', '39도시', '헬로팬닭', '홍대개미', '7번가피', '송파만다', '나인곱창', '써브웨이', '봉명동내', '폴트버거', '인생아구', '최가네한', '토핑폭탄', '김포불닭', '놀부 부', '박가부대', '랜돌프뉴', '잭아저씨', '짜글이가', '명작파스', '구래상회', '뜸들이다', '큰아들백', '워커스커', '라화방마', '깨돌이김', '직화삼겹', '춘리마라', '올데이파', '우리집반', '걸작떡볶', '유쾌한꼬', '피자스웨', '홍스족발', '원할머니', '킹프레소', '순살만공', '더팔당매', '보돌미역', '카페봄봄', '짱닭치킨', '국수나무', '땅끝수산', '꼬알라파', '대한냉면', '혼밥대왕', '갈비민족', '행복한찜', '황궁쟁반', '빕스얌딜', '안녕,닭', '병천토속', '맘스터치', '비오키친', '팔도한마', '올데이케', '상무초밥', '놀부보쌈', '여왕의브', '청춘식당', '삽교원조', '마라하오', '플러스8', '피제이피', '수제죽전', '탐나는피', '신자매김', '피자이탈', '프레드피', '모모타코', '울진죽변', '땡초곱창', '창타이누', '이화수육', '피자보이', '이토우야', '저팔계&', '피자프라', '전금례닭', '호랑이초', '공주닭발', '인생제육', 'GTS버', 'OK반점', '고기혁명', '미친피자', '천년닭강', '육회&연', '부성초밥', '초밥대통', '본점 장', '짚신스시', '대찌 대', '조형훈족', '빅스타피', '진지덮밥', '최고남제', '마라의신', '더티베지', '누보닭갈', '왕빈자삼', '감성커피', '콩샌전북', '윤희횟집', '무국적식

- 띄어쓰기 기준으로 음식점 이름 쪼개보기

In [ ]:
short_name2 = []
for store in corrected_store:
    tmp = store.split(' ')
    short_name2.extend(tmp)

short_name2 = list(set(short_name2)) # 중복 제거

In [ ]:
len(short_name2)

332

- 쪼개진 short_name 확인해서 음식점 이름으로 쓰일만한 것 남기기

In [ ]:
short_name2 = ['두리',
 '구구족',
 '랜돌프',
 '글이가',
 '팔당',
 '부성',
 '빈자',
 '베지',
 '시나',
 '알촌',
 '승도리네곱도리탕',
 '스웨이',
 '삼파전',
 '홍싸롱',
 '프레드',
 '빕스얌딜리버리',
 '피제이',
 '깨돌이',
 '소빅',
 '이토야',
 '춘리',
 '프라',
 '삽교',
 'GTS',
 '윤희',
 '1989마라탕',
 '써브웨이',
 '날제육',
 '냉면숯불고기전문점줄벼락',
 '탕화',
 '미락',
 '덮덮밥',
 '살만',
 '소우',
 '스시가',
 '폴트',
 '프레',
 '마라의신마라탕마라샹궈',
 '마라하오',
 '이태리면',
 '백순대',
 '이화수',
 '후스',
 '황궁쟁반짜장',
 '국수나무',
 '카산도',
 '박가',
 '덴버',
 '텔로',
 '고씨네',
 '우시장',
 '할머니보쌈',
 '독습',
 '오키친',
 '도야',
 '39도시락',
 '빽다방',
 '보도로',
 '라화방마라탕',
 '수비자',
 '뭇국적',
 '빅스타',
 '라이크',
 '죽변항',
 '손수경의',
 '용문',
 '깨비',
 '텐텐',
 '띵하오',
 '당치',
 '놀부보쌈족발',
 '1988응답하라',
 '최가네',
 '조형',
 '만다린',
 '7번가',
 '봄봄',
 '전금례닭볶음탕',
 '밈이',
 '쿵후',
 '누보',
 '글이',
 '워커스']

In [ ]:
store = tmp + corrected_store + tmp2 + short_name + short_name2

In [ ]:
store_list = list(set(store))
store_list

['스시사소',
 '박가부',
 '폴트버',
 '냉면&',
 '청년치킨',
 '코브라 독습',
 '우시장국',
 '스시가',
 '땅끝 수산',
 'GTS',
 '이끌림 마라탕',
 '코브라독',
 '탕화쿵푸',
 '떡형',
 '닭장수섭',
 '더팔당매운오징어닭발김치찜',
 '프레',
 '백순대',
 '나인 곱창',
 '과일에반',
 '본점장충왕족발보쌈',
 '박가 부대',
 '워커스커피로스터스',
 '이토',
 '봉명동내커피',
 '탐나는피자',
 '헬로팬닭갈비마약볶음밥',
 '냉면숯불고기전문점줄벼락',
 '자성당',
 '도우라이',
 '김포불',
 '윤희',
 '승도리네',
 '맘스터치대덕테크노벨리점',
 '쇼부덮밥',
 '홍스족',
 '두리',
 '프레드 피자',
 '이배달 ',
 '텐텐마라',
 '떡깨비',
 '삽교 원조 두리 곱창',
 '띵하',
 '걸작떡',
 '큰아들 백순대',
 '시민 보쌈족발 감자탕',
 '손수경의',
 '킹프',
 '백 소장',
 '스시 리',
 '플러스 82',
 '모어댄버터',
 '랜돌프뉴욕페페로니',
 '이끌림마',
 '띵하오차',
 '빅스타',
 '도야짬뽕',
 '부대찌개',
 '39도시',
 '헬로팬닭',
 '홍대개미',
 '공주닭',
 '팔도한마당순대',
 '7번가피',
 '더티베',
 '나인곱',
 '고씨네 카레',
 '송파만다',
 '무국적',
 '더팔당',
 '날제육',
 '수상한삼겹살',
 '텐텐마라탕',
 '배달의 쌀국수',
 '수제죽전문점',
 '누보 닭갈비',
 '나인곱창',
 '킹프레소빅와플',
 '조형 훈 족발보쌈',
 '써브웨이',
 '도우라이크피자',
 '롱쓰부대찌개',
 '봉명동내',
 '폴트버거',
 '띵하오차이',
 '수비자',
 '인생아구',
 '대한 냉면',
 '진지덮',
 '닭 장수 섭 삼계탕 전기구이 통닭',
 '마라하',
 '승부 덮밥',
 '구래 상회',
 '상무 초밥',
 '피자스웨그',
 '야키토',
 '당치땡',
 '이토우',
 '최가네한',
 '토핑폭탄',
 '부대찌개 자신 

- 빠진 음식점 이름 추가하기

In [ ]:
store_add = ['당신은 지금 치킨이 땡긴다','당신은 지금 치킨이 당긴다','마라 하오','빕스 얌 딜리버리','한식 끼','헬로팬','라화방','미.친피자','밈 피자','부자집','안녕닭','토핑폭탄','카마쿠라','땡초곱창','이배달','프루타','시민족발보쌈','조형훈의족발보쌈도시락','잭아저씨 족발보쌈','프라 텔로','홍스','도우 라이크']

In [ ]:
store_list.extend(store_add)

In [ ]:
train_df['store'] = train_df['corrected_reply'].apply(lambda x: next((store for store in store_list if store in x), " "))

In [ ]:
train_df.head()

,고객리뷰,corrected_reply,store
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 ...,
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해 드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것...,
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,


##5_지점명 키워드 컬럼화

In [ ]:
store_list = train['가게이름']
branch_arr = (store_list.str.split('-| ')).str.get(1).unique()
branch_arr

array(['방이점', '로스터스', '가락점', '헬리오시티점', '석촌점', '송파점', nan, '삼전점', '본점',
       '미아본점', '성신여대역점', '대학로점', '성북점', '동대문구점', '북서울꿈의숲점', '김치찜',
       '혜화막내딸지점', '대파생고기김치찌개&정육왕국물두루치기', '부천점', '부평점', '부천부평점', '본가진죽',
       '분식', '신중동점', '부천시청점', '도촌점', '야탑점', '판교점', '수제돈까스&파스타', '정자본점',
       '미금본점', '미금점', '분당정자점', '분당미금역점', '죽전점', '용인수지점', '카마쿠라', '서현본점',
       '분당점', '', '용암점', '마약삼겹떡볶이', '대전4호점', '신탄진점', '신일점', '목동충촌점',
       '대전동구점', '우송대점', '대전역점', '대흥문창점', '서대전점', '유성봉명점', '장충왕족발보쌈', '대전',
       '대덕테크노벨리점', '대전전민점', '마라탕', '대전한남대점', '충남대점', '유성점', '광주백운점',
       '계림점', '조대점', '광주본점', '광주광천점', '북구(전대)점', '광주직영점', '부대찌개', '전주평화점',
       '객사점', '전주1호점', '전주금암점'], dtype=object)

In [ ]:
branch_lst = [item for item in branch_arr if isinstance(item, str) and item.endswith('점')]
branch_lst

['방이점',
 '가락점',
 '헬리오시티점',
 '석촌점',
 '송파점',
 '삼전점',
 '본점',
 '미아본점',
 '성신여대역점',
 '대학로점',
 '성북점',
 '동대문구점',
 '북서울꿈의숲점',
 '혜화막내딸지점',
 '부천점',
 '부평점',
 '부천부평점',
 '신중동점',
 '부천시청점',
 '도촌점',
 '야탑점',
 '판교점',
 '정자본점',
 '미금본점',
 '미금점',
 '분당정자점',
 '분당미금역점',
 '죽전점',
 '용인수지점',
 '서현본점',
 '분당점',
 '용암점',
 '대전4호점',
 '신탄진점',
 '신일점',
 '목동충촌점',
 '대전동구점',
 '우송대점',
 '대전역점',
 '대흥문창점',
 '서대전점',
 '유성봉명점',
 '대덕테크노벨리점',
 '대전전민점',
 '대전한남대점',
 '충남대점',
 '유성점',
 '광주백운점',
 '계림점',
 '조대점',
 '광주본점',
 '광주광천점',
 '북구(전대)점',
 '광주직영점',
 '전주평화점',
 '객사점',
 '전주1호점',
 '전주금암점']

- 지점명에 띄어쓰기가 있을 경우

In [ ]:
branch_space = []
for name in branch_lst:
    hanspell_sent = spell_checker.check(name)
    branch_space.append(hanspell_sent.checked)

branch_space

['방이점',
 '가락점',
 '헬리오 시티점',
 '석촌점',
 '송파점',
 '삼전점',
 '본점',
 '미아 본점',
 '성신여대역점',
 '대학로점',
 '성북점',
 '동대문구점',
 '북서울꿈의 숲점',
 '혜화 막내딸 지점',
 '부천점',
 '부평점',
 '부천 부평점',
 '신중동점',
 '부천 시청점',
 '도촌점',
 '야탑점',
 '판교점',
 '정자본점',
 '미금 본점',
 '미금점',
 '분당 정자점',
 '분당 미금역점',
 '죽전점',
 '용인 수지점',
 '서현 본점',
 '분당점',
 '용암점',
 '대전 4호점',
 '신탄진점',
 '신일점',
 '목동충촌점',
 '대전 동구점',
 '우송대점',
 '대전역점',
 '대흥 문창점',
 '서대전점',
 '유성 봉명점',
 '대덕테크노벨리점',
 '대전 전민점',
 '대전 한남대점',
 '충남대점',
 '유성점',
 '광주백운점',
 '계림점',
 '조대점',
 '광주 본점',
 '광주 광천점',
 '북구(전대) 점',
 '광주 직영점',
 '전주 평화점',
 '객사점',
 '전주 1호점',
 '전주 금암점']

- 띄어쓰기 적용한 가게명 중 '점' 찾기 (ex- 콩 샌 전북대점)

In [ ]:
branch = []
for i in corrected_store:
    if i.endswith('점'):
        branch.append(i)
branch

['저팔계 중국 마라탕 덮밥 요리 전문점',
 '콩 샌 전북대점',
 '맘스터치 대덕테크노벨리점',
 'OK 반점',
 '이 배달 마약 삼겹 떡볶이 상당구점',
 '수제 죽 전문점',
 '탕화 쿵후 마라탕 가오점',
 '닭 강점',
 '도야 짬뽕 계림점']

In [ ]:
branch_list = branch_lst + branch_space

- 지점명 추가

In [ ]:
branch_add = ['전북대점','관평점','부천대점']

In [ ]:
branch_list.extend(branch_add)

In [ ]:
branch_list = list(set(branch_list)) # 중복 제거
branch_list

['대흥 문창점',
 '광주 직영점',
 '전주1호점',
 '혜화막내딸지점',
 '북구(전대) 점',
 '삼전점',
 '분당 미금역점',
 '헬리오시티점',
 '성신여대역점',
 '석촌점',
 '부평점',
 '송파점',
 '용암점',
 '미금 본점',
 '대흥문창점',
 '대전한남대점',
 '계림점',
 '부천 시청점',
 '미아본점',
 '성북점',
 '도촌점',
 '판교점',
 '전주금암점',
 '대전4호점',
 '서현 본점',
 '전주 평화점',
 '정자본점',
 '분당 정자점',
 '관평점',
 '대학로점',
 '신탄진점',
 '헬리오 시티점',
 '방이점',
 '신중동점',
 '부천점',
 '우송대점',
 '대전전민점',
 '광주직영점',
 '죽전점',
 '전주평화점',
 '유성 봉명점',
 '북구(전대)점',
 '유성봉명점',
 '미아 본점',
 '전북대점',
 '광주백운점',
 '대덕테크노벨리점',
 '신일점',
 '유성점',
 '미금점',
 '용인 수지점',
 '목동충촌점',
 '북서울꿈의숲점',
 '객사점',
 '부천 부평점',
 '전주 1호점',
 '대전 전민점',
 '동대문구점',
 '조대점',
 '광주 본점',
 '미금본점',
 '대전동구점',
 '분당미금역점',
 '혜화 막내딸 지점',
 '분당정자점',
 '광주 광천점',
 '광주본점',
 '서현본점',
 '대전 동구점',
 '광주광천점',
 '서대전점',
 '부천시청점',
 '대전역점',
 '대전 한남대점',
 '용인수지점',
 '부천부평점',
 '충남대점',
 '가락점',
 '분당점',
 '부천대점',
 '전주 금암점',
 '대전 4호점',
 '본점',
 '야탑점',
 '북서울꿈의 숲점']

In [ ]:
train_df['branch'] = train_df['corrected_reply'].apply(lambda x: next((branch for branch in branch_list if branch in x), " "))
train_df['branch']

0         
1         
2         
3         
4         
        ..
37135     
37136     
37137     
37138     
37139     
Name: branch, Length: 37140, dtype: object

In [ ]:
train_df.head()

,고객리뷰,corrected_reply,store,branch
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,,
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 ...,,
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해 드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것...,,
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,,
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,,


##6_음식 키워드 컬럼화

In [ ]:
food_list = pd.read_csv(f"{DATA_PATH}food.csv").drop_duplicates(ignore_index = True)["대표식품명"]
food_list

0          케이크
1          치즈빵
2         카스텔라
3          츄러스
4         치아바타
        ...   
753      햄채소볶음
754    무생채(무만)
755     참나물겉절이
756     치커리겉절이
757       짜장소스
Name: 대표식품명, Length: 758, dtype: object

In [ ]:
food_lst = []
for food in food_list:
    parts = re.split('/|-|\(', food)
    food_lst.append(parts[0])

In [ ]:
food_lst

['케이크',
 '치즈빵',
 '카스텔라',
 '츄러스',
 '치아바타',
 '와플',
 '찰떡빵',
 '김밥',
 '덮밥',
 '보리밥',
 '볶음밥',
 '비빔밥',
 '삼각김밥',
 '수수밥',
 '알밥',
 '영양돌솥밥',
 '오므라이스',
 '자장밥',
 '잡곡밥',
 '잡채밥',
 '주먹밥',
 '짬뽕밥',
 '차조밥',
 '찰밥',
 '초밥',
 '카레라이스',
 '콩나물밥',
 '콩밥',
 '하이라이스',
 '현미밥',
 '회덮밥',
 '가래떡',
 '경단',
 '꿀떡',
 '모듬찰떡',
 '백설기',
 '송편',
 '시루떡',
 '약식',
 '인절미',
 '절편',
 '증편',
 '찹쌀떡',
 '도넛',
 '마늘빵',
 '머핀',
 '모닝빵',
 '모카빵',
 '베이글',
 '샌드위치',
 '소보로빵',
 '식빵',
 '초코소라빵',
 '크로와상',
 '파이',
 '페이스트리',
 '피자',
 '피자빵',
 '햄버거',
 '호떡',
 '간자장',
 '국수',
 '냉면',
 '떡국',
 '떡만두국',
 '라면',
 '만두',
 '만두국',
 '메밀국수',
 '수제비',
 '스파게티',
 '우동',
 '우동볶음',
 '월남쌈',
 '자장면',
 '짬뽕',
 '쫄면',
 '칼국수',
 '콩국수',
 '닭죽',
 '스프',
 '오리죽',
 '전복죽',
 '채소죽',
 '팥죽',
 '호박죽',
 '흑임자죽',
 '갈비탕',
 '감자국',
 '곰탕',
 '굴국',
 '김치국',
 '내장탕',
 '냉국',
 '달걀국',
 '달걀탕',
 '닭곰탕',
 '도가니탕',
 '된장국',
 '무국',
 '묵국',
 '미역국',
 '배추국',
 '북어국',
 '사골국',
 '삼계탕',
 '새우탕',
 '설렁탕',
 '순대국',
 '어묵국',
 '오징어국',
 '육개장',
 '장어탕',
 '재첩국',
 '조개국',
 '콩나물국',
 '토란국',
 '해장국',
 '갈치찌개',
 '감자탕',
 '김치전골',
 '김치찌개',
 '꽃게탕',
 '낙지전골',
 '동태국',
 '동

In [ ]:
food_tmp = []
for name in food_lst:
    hanspell_sent = spell_checker.check(name)
    food_tmp.append(hanspell_sent.checked)

food_tmp

['케이크',
 '치즈빵',
 '카스텔라',
 '추로스',
 '치아바타',
 '와플',
 '찰떡 빵',
 '김밥',
 '덮밥',
 '보리밥',
 '볶음밥',
 '비빔밥',
 '삼각김밥',
 '수수밥',
 '알밥',
 '영양 돌솥밥',
 '오므라이스',
 '자장밥',
 '잡곡밥',
 '잡채밥',
 '주먹밥',
 '짬뽕밥',
 '차조밥',
 '찰밥',
 '초밥',
 '카레라이스',
 '콩나물밥',
 '콩밥',
 '하이라이스',
 '현미밥',
 '회덮밥',
 '가래떡',
 '경단',
 '꿀떡',
 '모둠 찰떡',
 '백설기',
 '송편',
 '시루떡',
 '약식',
 '인절미',
 '절편',
 '증편',
 '찹쌀떡',
 '도넛',
 '마늘빵',
 '머핀',
 '모닝빵',
 '모카빵',
 '베이글',
 '샌드위치',
 '곰보빵',
 '식빵',
 '초코 소라고 빵',
 '크루아상',
 '파이',
 '페이스트리',
 '피자',
 '피자빵',
 '햄버거',
 '호떡',
 '간자장',
 '국수',
 '냉면',
 '떡국',
 '떡만둣국',
 '라면',
 '만두',
 '만둣국',
 '메밀국수',
 '수제비',
 '스파게티',
 '가락국수',
 '가락국수 볶음',
 '월남쌈',
 '자장면',
 '짬뽕',
 '쫄면',
 '칼국수',
 '콩국수',
 '닭죽',
 '수프',
 '오리 죽',
 '전복죽',
 '채소 죽',
 '팥죽',
 '호박죽',
 '흑임자죽',
 '갈비탕',
 '감잣국',
 '곰탕',
 '굴국',
 '김칫국',
 '내장탕',
 '냉국',
 '달 결국',
 '달걀탕',
 '닭곰탕',
 '도가니탕',
 '된장국',
 '뭇국',
 '묵국',
 '미역국',
 '배춧국',
 '북엇국',
 '사골국',
 '삼계탕',
 '새우탕',
 '설렁탕',
 '순댓국',
 '어묵국',
 '오징어 국',
 '육개장',
 '장어탕',
 '재첩 국',
 '조갯국',
 '콩나물국',
 '토란국',
 '해장국',
 '갈치 찌개',
 '감자탕',
 '김치전골',
 '김치찌개',
 '꽃게탕',
 '낙지

In [ ]:
food_short = []
for food_space in food_tmp:
    tmp = food_space.split(' ')
    food_short.extend(tmp)

food_short = list(set(food_short)) # 중복 제거
food_short

['새우탕',
 '떡국',
 '고추튀김',
 '새우튀김',
 '깻잎나물볶음',
 '김',
 '어탕',
 '수프',
 '갈치구이',
 '전복탕',
 '새우전',
 '미꾸라지',
 '도라지나물',
 '호박죽',
 '기피',
 '밤',
 '어묵조림',
 '조개젓',
 '갑오징어무침',
 '북어포',
 '닭발볶음',
 '오리고기로',
 '도넛',
 '우럭',
 '곤약',
 '배춧국',
 '에이드',
 '순두부찌개',
 '닭발구이',
 '완자전',
 '게살죽',
 '굴비구이',
 '양파링',
 '꽁치찌개',
 '돼지갈비찜',
 '라면',
 '북어조림',
 '달걀',
 '꽁치조림',
 '애호박볶음',
 '멸치젓',
 '닭죽',
 '오이무침',
 '해신탕',
 '화양적',
 '삼각김밥',
 '미역줄기볶음',
 '미역튀각',
 '꼬치구이',
 '김치찌개',
 '오므라이스',
 '김칫국',
 '콩나물국',
 '낙지전골',
 '오징어찌개',
 '수수팥떡',
 '과ㆍ체',
 '스테이크',
 '우유',
 '소갈비찜',
 '부대찌개',
 '쥐포',
 '파래무침',
 '소탕',
 '순대전골',
 '지리',
 '북어찜',
 '조기조림',
 '들깨국',
 '고춧잎나물',
 '호박오가리나물',
 '가스',
 '콩나물밥',
 '인절미',
 '피조개',
 '뱅쇼',
 '조기',
 '카레라이스',
 '런천미트',
 '깐풍',
 '프레즐',
 '초콜릿',
 '전',
 '누룽지',
 '샌드위치',
 '깐쇼새우',
 '참나물겉절이',
 '쑥갓나물',
 '베이글',
 '순대볶음',
 '게국지',
 '아이스티',
 '천',
 '된장무침',
 '밀면',
 '참치',
 '김치찜',
 '수수밥',
 '고추',
 '달',
 '유채나물',
 '미음',
 '얼갈이',
 '다식',
 '풋고추찜',
 '약과',
 '풋고추볶음',
 '붕어찜',
 '레몬홍차',
 '발',
 '육회',
 '오이지무침',
 '얼갈이배추김치',
 '다쿠아즈',
 '미트볼',
 '연어구이',
 '버섯찌개',
 '칼국수',
 '떡',
 '호박전'

In [ ]:
len(food_short)

753

In [ ]:
drop_name = ['대구','영양','소라고','모둠','조기','청자','결국','내장','다리','귀이','매작과']

In [ ]:
food_short = [i for i in food_short if i not in drop_name]

In [ ]:
len(food_short)

742

In [ ]:
food = food_lst + food_tmp + food_short

In [ ]:
food_lst = list(set(food)) # 중복 제거
food_lst

['새우탕',
 '떡국',
 '고추튀김',
 '깻잎나물볶음',
 '새우튀김',
 '김',
 '어탕',
 '숙주미나리나물',
 '두부 전',
 '들깻잎 장아찌',
 '수프',
 '갈치구이',
 '전복탕',
 '새우전',
 '미꾸라지',
 '도라지나물',
 '유채물김치',
 '오겹살구이',
 '소고기샤브샤브',
 '호박죽',
 '기피',
 '불낙 전골',
 '버섯조림',
 '밤',
 '어묵조림',
 '조개젓',
 '꼬막 찜',
 '갑오징어무침',
 '북어포',
 '달걀국',
 '닭발볶음',
 '오리고기로',
 '채소 튀김',
 '중식 잡채',
 '도넛',
 '우럭',
 '곤약',
 '배춧국',
 '에이드',
 '순두부찌개',
 '닭발구이',
 '채소 죽',
 '열무 얼갈이김치',
 '완자전',
 '소고기 산적',
 '파래 전',
 '게살죽',
 '오리고기로스',
 '다슬기 국 수',
 '굴비구이',
 '양파링',
 '꽁치찌개',
 '햄 채소볶음',
 '미나리전',
 '돼지갈비찜',
 '소내장구이',
 '라면',
 '북어국',
 '유채 물김치',
 '치킨 데리야키',
 '북어조림',
 '우거지 나물 무침',
 '달걀',
 '기타 차',
 '꽁치조림',
 '애호박볶음',
 '멸치젓',
 '참치덮밥',
 '옥수수 구이',
 '닭죽',
 '오이무침',
 '깐풍 새우',
 '해신탕',
 '화양적',
 '삼각김밥',
 '미역줄기볶음',
 '양파링튀김',
 '미역튀각',
 '소고기 볶음',
 '꼬치구이',
 '김치찌개',
 '오므라이스',
 '콩나물국',
 '오징어찌개',
 '낙지전골',
 '김칫국',
 '수수팥떡',
 '초코 소라고 빵',
 '과ㆍ체',
 '마늘쫑무침',
 '스테이크',
 '우유',
 '소갈비찜',
 '부대찌개',
 '오징어 국',
 '배추 나물',
 '파래무침',
 '쥐포',
 '소탕',
 '순대전골',
 '지리',
 '북어찜',
 '조기조림',
 '톳 나물',
 '소라산적',
 '오징어 조림',
 '호박오가리나물 무침',
 '소고기 샤부샤부',
 '꼬치구

In [ ]:
all_food = [word for word in food_lst if len(word) > 1]

- 빠진 음식이름 추가

In [ ]:
food_add = ['펩시콜라','똠얌꿍','닭','목살','얌꿍','꽈리고추','파스타']

In [ ]:
all_food.extend(food_add)

In [ ]:
all_food.remove('파이')

In [ ]:
if '닭' in all_food:
    print("True")
else:
    print("False")

True


In [ ]:
train_df['food'] = train_df['corrected_reply'].apply(lambda x: next((food for food in all_food if food in x), " "))

In [ ]:
train_df.head()

,고객리뷰,corrected_reply,store,branch,food
0,맛있게 잘먹었습니다 묵사발도 시원하니 맛있네요,즐거운 순간을 저희와 함께해 주셔서 감사합니다 보람차게 일할 수 있게 해 주셔서 감...,,,
1,기름 있는 부분이 생각보다 많아요! 참고하세요,소중한 의견 남겨 주셔서 감사합니다. 잘 새겨듣고 앞으로 더욱더 개선할 수 있도록 ...,,,
2,양도 푸짐하고 맛있습니다! 청국장 냄새 많이 안나고 콩도 많이 들어있어서 좋았습니다!,늘 최고의 메뉴를 제공해 드리고 싶습니다! 맛있는 메뉴를 넉넉하게 먹을 수 있는 것...,,,
3,".,.",저희 매장과 함께 편안한 시간 되셨을까요? 1. 고객을 생각하는 마음 2. 더 나은...,,,
4,냉면을 따로 시키고 미니냉면을 합친것인데 양이 좀 적어보이고. 김치정식도 양이 적어...,이번 주문을 통해 한걸음 더 가까워진 것 같아 기분이 좋아요! 고객님의 행복한 시간...,,,


# 긍/부정 비율 확인

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
from transformers import pipeline

def analyze_reviews(reviews):
    # 모델 불러오기
    model_name = "MLOpsEngineer/review-rate-prediction"
    classifier = pipeline("sentiment-analysis", model=model_name)

    results = []
    for review_text in tqdm(reviews):
        # 모델 추론
        result = classifier(review_text)[0]

        # 결과 해석
        label = result['label']
        score = result['score']

        if label in ['LABEL_0', 'LABEL_1']:
            sentiment = 0 # '부정'
        else:
            sentiment = 1 # '긍정'

        results.append((review_text, sentiment, score))

    # 데이터프레임 생성
    tmp = pd.DataFrame(results, columns=['review', 'sentiment', 'score'])
    return tmp

In [ ]:
sentiment_train = analyze_reviews(train['고객리뷰'])
sentiment_train

- Sentiment(긍/부정 사전학습 결과) 비율

In [ ]:
for i in range(0, 2):
    num_rating = sentiment_train[sentiment_train['sentiment'] == i].shape[0]
    print(f"별점 {i}점 개수: {num_rating}, {num_rating / sentiment_train.shape[0] * 100}%")

In [ ]:
sentiment_train = sentiment_train[['sentiment','score']]

In [ ]:
train = train[['고객리뷰','별점','사장댓글']]
train.columns = ['review','rating','reply']

In [ ]:
train = pd.concat([train, sentiment_train],axis=1)

# 부정데이터 오버샘플링

- 기존데이터의 부정 리뷰수: 1824개

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00


In [ ]:
import os
import openai

In [ ]:
train['reply'] = reply_list
train.head()
mask = (train['review'].str.len() > 10 ) & (train['rating'] == 5) & (train['sentiment'] == 1) & (train['score'] > 0.7 )
df = train[mask]
df.shape
# 챗 gpt로 부정문 가져오기
pos_list = df['review']

##1_긍정문 -> 부정문 변경

In [ ]:
import os
import openai

def get_neg_sentence(n,m):
    openai.api_key = 'API'

    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {
                'role': 'user',
                'content': f'{pos_list[n:m]} 이 10개의 긍정적인 리뷰 문장들을, 내용은 동일하되 부정문으로만 바꿔서 순서대로 리스트에 담아줘. ["음식이 다 식어서 와서 눅눅해졌어요.. 실망이에요 다신 안시킬 거 같아요" , ...] 식으로. 10개의 부정문을 모두 잘리지않게 리스트 형태에 담아줘.'
            }
        ]
    )
    globals()[f"postoneg_review_{n}_{m}"] = completion.choices[0].message.content

In [ ]:
from tqdm import tqdm

num = 11
for i in tqdm(range(num*1000,(num+1)*1000,10)):
    get_neg_sentence(i,i+10)

In [ ]:
# 전체 변환 리뷰의 모양 확인

combined_negative_reviews = []

# for 루프를 사용하여 리뷰를 하나로 결합합니다.
for i in tqdm(range(num*100+1, (num+1)*100+1)):  # 1부터 100까지의 범위를 설정 1~101,  / 1000_1010까지 생성완료 / 1001 / 101, 102
    # 리스트의 이름을 동적으로 생성하여 가져옵니다.
    list_name = f"postoneg_review_{i * 10 - 10}_{i * 10}"  # 리스트 이름 예: postoneg_review_
    current_list = globals()[list_name]  # 동적으로 리스트를 가져옵니다.

    # 현재 리스트를 combined_negative_reviews에 결합합니다.
    combined_negative_reviews.append(current_list)

# 모든 리뷰가 결합된 결과를 확인합니다.
print(combined_negative_reviews)

In [ ]:
filtered_pos_train = df
filtered_pos_train['neg_review'] =0

- 추출

In [ ]:
import numpy as np

for i in range(num*1000,(num+1)*1000,10):
    # 변수 이름 생성
    var_name = f"postoneg_review_{i}_{i + 10}"

    # 해당 변수에서 리뷰 추출 및 분할하여 리스트에 추가
    reviews = globals()[var_name].split('\n')

    # 슬라이스의 길이와 할당하려는 값의 길이가 다른 경우 NaN으로 채움
    if len(reviews) ==  10:
        filtered_pos_train['neg_review'].iloc[i:i+10] = reviews

In [ ]:
for i in range(num*1000,(num+1)*1000,10):
    # 변수 이름 생성
    var_name = f"postoneg_review_{i}_{i + 10}"

    # 해당 변수에서 리뷰 추출 및 분할하여 리스트에 추가
    reviews_str = globals()[var_name]
    reviews = re.findall(r'"(.*?)"|\'(.*?)\'', reviews_str)

    # 슬라이스의 길이와 할당하려는 값의 길이가 다른 경우 NaN으로 채움
    if len(reviews) ==  10:
        filtered_pos_train['neg_review'].iloc[i:i+10] = reviews

In [ ]:
filtered_pos_train[num*1000:(num+1)*1000]

##2_부정리뷰에서 사장님 답변 얻기

In [ ]:
neg_list = filtered_pos_train['neg_review']

In [ ]:
# 부정변환 문장에 대한 사장님의 답변
def get_neg_reply(n, m):
    openai.api_key = 'API'

    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {
                'role': 'user',
                'content': f'{neg_list[n:m]} 이 10개의 부정적인 리뷰 문장들 각각에 대한 최대한 적절하고, 구체적이고 자연스러운 사장님의 한글 답변 10개를 최대한 길게 생성해서 순서대로 리스트에 담아줘. 각 문장은 " " 사이에, 콤마로 서로 연결시켜줘. 답변만 리스트에 담아줘.'
            }
        ]
    )
    globals()[f"postoneg_reply_{n}_{m}"] = completion.choices[0].message.content

In [ ]:
filtered_pos_train['neg_reply'] = 0

In [ ]:
from tqdm import tqdm

for i in tqdm(range(num*1000,(num+1)*1000,10)):
    get_neg_reply(i,i+10)

In [ ]:
# 전체 변환 리뷰의 모양 확인

combined_negative_replies = []

# for 루프를 사용하여 리뷰를 하나로 결합합니다.
for i in tqdm(range(num*100+1, (num+1)*100+1)):  # 1부터 100까지의 범위를 설정
    # 리스트의 이름을 동적으로 생성하여 가져옵니다.
    list_name = f"postoneg_reply_{i * 10 - 10}_{i * 10}"  # 리스트 이름 예: postoneg_review_
    current_list = globals()[list_name]  # 동적으로 리스트를 가져옵니다.

    # 현재 리스트를 combined_negative_reviews에 결합합니다.
    combined_negative_replies.append(current_list)

# 모든 리뷰가 결합된 결과를 확인합니다.
print(combined_negative_replies)

- 추출

In [ ]:
import numpy as np

for i in range(num*1000,(num+1)*1000,10):
    # 변수 이름 생성
    var_name = f"postoneg_reply_{i}_{i + 10}"

    # 해당 변수에서 리뷰 추출 및 분할하여 리스트에 추가
    replies = globals()[var_name].split('\n')

    # 슬라이스의 길이와 할당하려는 값의 길이가 다른 경우 NaN으로 채움
    if len(replies) ==  10:
        filtered_pos_train['neg_reply'].iloc[i:i+10] = replies

In [ ]:
for i in range(num*1000,(num+1)*1000,10):
    # 변수 이름 생성
    var_name = f"postoneg_reply_{i}_{i + 10}"

    # 해당 변수에서 리뷰 추출 및 분할하여 리스트에 추가
    replies_str = globals()[var_name]
    replies = re.findall(r'"(.*?)"|\'(.*?)\'', replies_str)

    # 슬라이스의 길이와 할당하려는 값의 길이가 다른 경우 NaN으로 채움
    if len(replies) ==  10:
        filtered_pos_train['neg_reply'].iloc[i:i+10] = replies

In [ ]:
mask = (filtered_pos_train['neg_review'] != 0) & (filtered_pos_train['neg_reply'] != 0)
filtered_final_train = filtered_pos_train[mask]

In [ ]:
filtered_final_train

In [ ]:
filtered_final_train.to_csv(f"filtered_final_train_10000_11000.csv")

- 오버샘플링 데이터 합치기

In [ ]:
# num0~8,11은 기본 답변 / num9~10은 최대한 길게 답변 요청

gpt_aug_num0 = pd.read_csv("filtered_final_train_1_1000.csv") #850
gpt_aug_num1 = pd.read_csv("filtered_final_train_1000_2000.csv") #860
gpt_aug_num2 = pd.read_csv("filtered_final_train_2000_3000_2.csv") #775
gpt_aug_num3 = pd.read_csv("filtered_final_train_3000_4000.csv") #950
gpt_aug_num4 = pd.read_csv("filtered_final_train_4000_5000.csv") #940

gpt_aug_num5 = pd.read_csv("filtered_final_train_5000_6000.csv") #920
gpt_aug_num6 = pd.read_csv("filtered_final_train_6000_7000.csv") #900
gpt_aug_num7 = pd.read_csv("filtered_final_train_7000_8000.csv") #920
gpt_aug_num8 = pd.read_csv("filtered_final_train_8000_9000.csv") #830
gpt_aug_num9 = pd.read_csv("filtered_final_train_9000_10000.csv") #815
gpt_aug_num10 = pd.read_csv("filtered_final_train_10000_11000.csv") #840
gpt_aug_num11 = pd.read_csv("filtered_final_train_11000_12000.csv") #745

In [ ]:
tmp = pd.concat([gpt_aug_num0,gpt_aug_num1,gpt_aug_num2, gpt_aug_num3, gpt_aug_num4, gpt_aug_num5, gpt_aug_num6, gpt_aug_num7, gpt_aug_num8, gpt_aug_num9, gpt_aug_num10, gpt_aug_num11],axis=0)
tmp

##3_데이터 정제

###영어 및 특수문자 제거

In [ ]:

review_list = []
for review in tmp['neg_review']:
    if isinstance(review, str):

        pat = re.compile("[^가-힣0-9.,~?!]") # 영어 제거
        result = pat.sub(" ", review)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

tmp['neg_review'] = review_list
tmp.head()


In [ ]:
reply_list = []
for reply in tmp['neg_reply']:
    if isinstance(reply, str):

        pat = re.compile("[^가-힣0-9.,~?!]") # 영어 제거
        result = pat.sub(" ", reply)

        result = re.sub(r',\s*$', '', result)
        result = re.sub(r'^\s*,', '', result)
        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

tmp['neg_reply'] = reply_list
tmp.head()


- 2차 정제

In [ ]:
review_list = []
for review in tmp['neg_review']:
    if isinstance(review, str):
        pat = r'^\s*,\s*'
        result = re.sub(pat, '', review)
        review_list.append(result)
    else:
        review_list.append("")

tmp['neg_review'] = review_list
tmp.head()

In [ ]:
reply_list = []
for reply in tmp['neg_reply']:
    if isinstance(reply, str):
        pat = r'^\s*,\s*'
        result = re.sub(pat, '', reply)
        reply_list.append(result)
    else:
        reply_list.append("")

tmp['neg_reply'] = reply_list
tmp.head()

In [ ]:
review_list = []
for review in tmp['neg_review']:
    if isinstance(review, str):
        pat = r'^\s*'
        result = re.sub(pat, '', review)
        review_list.append(result)
    else:
        review_list.append("")

tmp['neg_review'] = review_list
tmp.head()

In [ ]:
reply_list = []
for reply in tmp['neg_reply']:
    if isinstance(reply, str):
        pat = r'^\s*'
        result = re.sub(pat, '', reply)
        reply_list.append(result)
    else:
        reply_list.append("")

tmp['neg_reply'] = reply_list
tmp.head()

In [ ]:
# 결측치 확인
tmp = tmp.drop(columns = 'Unnamed: 0')
tmp.isna().sum()

###짧은 데이터 제거

In [ ]:
mask = (tmp['neg_review'].str.len() <= 4) | (tmp['neg_reply'].str.len() <= 4)
tmp[mask]

### 특이 패턴 제거

In [ ]:
index_error = tmp[tmp['neg_reply'].str.contains("답변이 없습니다")].index
tmp.drop(index =index_error, inplace=True )

##4_키워드 컬럼 추가

- food 컬럼 추가

In [ ]:
tmp['food'] = tmp['neg_reply'].apply(lambda x: next((food for food in all_food if food in x), " "))

- store & branch 컬럼 추가

In [ ]:
tmp['store'] = ''
tmp['branch'] = ''
tmp.head()

In [ ]:
tmp.columns = ['고객리뷰','사장댓글','sentiment','food','store','branch']

In [ ]:
tmp.to_csv(f'{DATA_PATH}gpt_9300.csv', index=False, encoding = "utf-8-sig")

# 데이터셋 합치기

In [ ]:
sentiment_train = sentiment_train[['sentiment']]

In [ ]:
df = pd.concat([train_df, sentiment_train], axis=1)
df.head()

In [ ]:
df = df[['고객리뷰','corrected_reply','store','branch','food','sentiment']]
df

- 긍부정 비율 조정 : 총 23,762개

In [ ]:
df = df.loc[df.sentiment == 1]
df = df.sample(frac=1, random_state=SEED)

In [ ]:
df = df[:11881]

In [ ]:
yogiyo_train = pd.concat([tmp,df], axis=0)
yogiyo_train

In [ ]:
for i in range(0, 2):
    num_rating = yogiyo_train[yogiyo_train['sentiment'] == i].shape[0]
    print(f"긍/부정 {i}점 개수: {num_rating}, {num_rating / yogiyo_train.shape[0] * 100}%")

In [ ]:
yogiyo_train.to_csv(f'{DATA_PATH}train_0921.csv', index=False, encoding = "utf-8-sig")